#### 사용자 정의 데이터셋
 - Pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하 (H/W, S/W) 및 많은 시간 소요에 대한 해결책으로 제시
 - 대량 데이터셋 전용 처리 모듈 제공
 - Dataset과 DataLoader 
    * Dataset       => 데이터전처리, 텐서화 등의 작업 진행 
    * DataLoader    => dataset 인스턴스를 사용해서 배치크기만큼ㄴ 데이터를 추출함 

[1]모듈로딩 및 데이터 준비 <hr>

In [39]:
# 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder  # 타게ㅐㅅ 컬럼 수치화

In [40]:
# 데이터
DATA_FILE = 'C:\Git\KDT\BigData\ML_Work\data\iris.csv'

In [41]:
irisDF = pd.read_csv(DATA_FILE)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [42]:
# 타겟 칼럼 수치화 => LabelEncoder
encoder = LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety'] = encoder.transform(irisDF['variety'])

In [43]:
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


[2] 사용자 정의 데이터셋 클래스 생성 <hr>

In [44]:
# --------------------------------------------------------------
# 클래스목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스이름 : CustomDataSet 
# 부모클래스 : torch.utils.Dataset
# 매개  변수 : featureDF, targetDF
# --------------------------------------------------------------

In [45]:
class CustomDataSet(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[-1]

    # 데이터의개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메ㅓㅅ드
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor( self.featureDF.iloc[idx].values )
        targetTS = torch.FloatTensor( self.targetDF.iloc[idx].values )
        return featureTS, targetTS

In [46]:
# ------------------------------------------------------------------
# 클래스기능 : 파일 확장자별 데이터프레임 변환 기능
# 클래스이름 : convertDataFrame
# 매개  변수 : data_path
# 함수  결과 : DataFrame
# ------------------------------------------------------------------
def convertDataFrame(data_path, exit_header=0):
    ext = data_path.rsplit('.')[-1]
    if ext == 'csv':
        return pd.read_csv(data_path, header = 0 if exit_header else None)
    elif ext == 'json':
        return pd.read_json(data_path)
    elif ext in ['xlsx', 'xls']:
        return pd.read_excel(data_path)
    else:
        return pd.read_table(data_path)

In [47]:
# ------------------------------------------------------------------
# 클래스 기능 : 파일기반 데이터셋
# 클래스이름 : FileDataset
# 매개  변수 : data_path 파일 경로
# 부모클래스 :  utils.data.Dataset
# ------------------------------------------------------------------
class FileDataSet(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, data_path):
        super().__init__()
        dataDF = convertDataFrame(data_path)
        self.featureDF = dataDF[dataDF.columns[:-1]]
        self.targetDF = dataDF[dataDF.columns[-1:]]
        self.n_features = dataDF.shape[1]
        self.n_rows = self.featureDF.shape[0]


    # 데이터의개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메ㅓㅅ드
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor( self.featureDF.iloc[idx].values )
        targetTS = torch.FloatTensor( self.targetDF.iloc[idx].values )
        return featureTS, targetTS

[3] 데이터셋 인스턴스 생성 <hr>

In [48]:
# 데이터 전처리
featureDF, targetDF = irisDF[irisDF.columns[:-1]], irisDF[[irisDF.columns[-1]]]

print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')
# irisDF

featureDF => (150, 4), targetDF => (150, 1)


In [49]:
# IRIS 데이터셋 인스턴스 생성
irisDS = CustomDataSet(featureDF, targetDF)

In [50]:
# IRIS 데이터셋 속성
irisDS.n_features, irisDS.n_rows

(4, 150)

In [51]:
# IRIS 데이터셋 메서드
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [52]:
## 필요한 것 : Dataset 인스턴스, Batch_size
irisDL = DataLoader(irisDS,batch_size=3)

In [53]:
for _ in irisDL:
    print(_)
    break

[tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000]]), tensor([[0.],
        [0.],
        [0.]])]


In [54]:
for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)
    break

torch.Size([3, 4]) torch.Size([3, 1])


In [55]:
dataDF = convertDataFrame(DATA_FILE)
dataDF.head()

,0,1,2,3,4
0,sepal.length,sepal.width,petal.length,petal.width,variety
1,5.1,3.5,1.4,.2,Setosa
2,4.9,3,1.4,.2,Setosa
3,4.7,3.2,1.3,.2,Setosa
4,4.6,3.1,1.5,.2,Setosa


## 사용자 정의 데이터셋
 - pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하 (H/w), (S/W) 및 많은 시간 소요에 따흔 해결책으로 제시
 - 대량 데티어셋 전용 처리 모듈 제공

 - iris.csv ==> 사용자 정의 데이터셋
 - DNN 모델 => 사용자 정의 모델


[1] 모듈 로딩 및 데이터 준비 <hr>

In [ ]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchmetrics.classification import F1Score
# from torchmetrics
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.

In [ ]:
## 필요한 것 : Dataset 인스턴스


[5] 

In [ ]:
#
# 모델 이름 : CustomModel
# 부모클래스 : nn.Module
# 매개 변수 : in_in, out_out, h_inout, h_cnt
# 모델구조
# - 입력층 : 입력 4개 출력 10개     AF ReLU -> LeakyReLU
# - 은닉층 : 입력 10개 출력 30개    AF ReLU -> LeakyReLU
# - 출력층 : 입력 30개 출력 1개     AF 분류 - 다중 SoftMax

class CustomDataSet(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(4,10)
        self.hidden_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,1)


    # 순방향 학습 메서드
    def forward(self, x):
        y = F.relu(self.in_layer(x))
        y = F.relu(self.hidden_layer(x))

        return F.softmax(self.out_layer(y))

        # y = F.relu(self.out_layer(x))

In [ ]:
## 모델 인스터늣 생성
model = CustomModel()


[6] 학습 <hr>

In [56]:
## 배치크기만큼 데이터와 타겟 추출해서 학습 진행
for dataTS, targetTS in irisDL:

    # 배치크기만큼의 학습 데이터
    print(dataTS.shape, targetTS.shape)

    # 배치크기만큼 학습 진행
    pre_y = model(dataTS)
    print(pre_y.shape, targetTS.shape)

    # 손실 계산
    loss = nn.CrossEntropyLoss()(pre_y,targetTS)

    break

torch.Size([3, 4]) torch.Size([3, 1])


NameError: name 'model' is not defined